In [8]:
import requests
import json
import forecastio
from pprint import pprint
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Nashville_airbnb import Nashville_airbnb
import datetime as dt

In [9]:
api_key = "a36c8345cc74c469a50078545e4970de"

In [10]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Nashville, TN")

NV_coord = []
NV_coord.append(location.latitude)
NV_coord.append(location.longitude)

NV_coord

[36.1622296, -86.7743531]

In [11]:
forecast = forecastio.load_forecast(api_key, NV_coord[0], NV_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Rain today through Thursday, with high temperatures bottoming out at 46°F on Monday.
rain
54.77


In [12]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28']

In [13]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [14]:
Nashville_airbnb.dtypes

Date         object
Under $50     int64
$50-100       int64
$100-150      int64
$150-200      int64
$200-250      int64
dtype: object

In [15]:
NV_table = pd.merge(df, Nashville_airbnb, on="Date")
NV_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-21,42.36,29.07,0.91,0.0049,rain,120,638,597,580,592
1,2018-12-22,50.55,40.17,0.06,0.0003,rain,116,564,562,557,571
2,2018-12-23,47.45,30.92,0.59,0.0084,rain,152,912,801,469,469
3,2018-12-24,45.59,32.70,0.04,0.0001,rain,155,992,719,696,351
4,2018-12-25,54.77,36.01,0.03,0.0001,rain,147,973,702,692,353
5,2018-12-26,59.74,50.95,0.00,0.0000,none,147,892,672,722,347
6,2018-12-27,65.08,53.86,0.91,0.1015,rain,137,698,564,390,256
7,2018-12-28,63.06,41.54,0.25,0.0015,rain,109,339,279,309,236
